# <span style="color:purple">Deep_Session_4차시_CNN 기초</span>

In [1]:
import warnings
warnings.simplefilter('ignore')

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets

- [Convolution의 output 크기](#Convolution의-output-크기-실습)
- [CNN 실습](#CNN-실습)
- [Data Augmentation 기법](#Data-Augmentation-기법-이용)

# Convolution의 output 크기 실습

In [26]:
nn.Conv2d(1,1,5)

Conv2d(1, 1, kernel_size=(5, 5), stride=(1, 1))

In [27]:
nn.Conv2d(1,1,(5,10), stride=(1,2), padding=(2,4))

Conv2d(1, 1, kernel_size=(5, 10), stride=(1, 2), padding=(2, 4))

In [28]:
# 예제 1
conv = nn.Conv2d(1,1,11, stride=4, padding=0)
print(conv)
inputs = torch.Tensor(1,1,227,227)
print(inputs.shape)
out = conv(inputs)
print(out.shape)

Conv2d(1, 1, kernel_size=(11, 11), stride=(4, 4))
torch.Size([1, 1, 227, 227])
torch.Size([1, 1, 55, 55])


In [29]:
# 예제 2
conv = nn.Conv2d(1,1,5, stride=1, padding=2)
print(conv)
inputs = torch.Tensor(1,1,32,32)
print(inputs.shape)
out = conv(inputs)
print(out.shape)

Conv2d(1, 1, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
torch.Size([1, 1, 32, 32])
torch.Size([1, 1, 32, 32])


In [30]:
# 예제 3
conv = nn.Conv2d(1,1,3, stride=1, padding=1)
print(conv)
inputs = torch.Tensor(1,1,64,32)
print(inputs.shape)
out = conv(inputs)
print(out.shape)

Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
torch.Size([1, 1, 64, 32])
torch.Size([1, 1, 64, 32])


# CNN 실습

In [2]:
# Module import하기

import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets

In [3]:
# 장비 확인하기
# DEVICE 설정
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')

print('Using PyTorch version:', torch.__version__, ' Device:', DEVICE)

Using PyTorch version: 1.11.0  Device: cuda


In [4]:
# 코드 내 하이퍼파리미터

BATCH_SIZE = 32
EPOCHS = 10

In [5]:
# CIFAR10 데이터 다운로드(Train set, Test set 분리하기)

train_dataset = datasets.CIFAR10(root = "../data/CIFAR_10",
                                train = True,
                                download = True,
                                transform = transforms.ToTensor())

test_dataset = datasets.CIFAR10(root = "../data/CIFAR_10",
                                train = False,
                                transform = transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = BATCH_SIZE,
                                          shuffle = True)

test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                         batch_size = BATCH_SIZE,
                                         shuffle = False)

Files already downloaded and verified


In [6]:
# 데이터 확인 1

for (X_train, y_train) in train_loader:
    print('X_train:', X_train.size(), 'type:', X_train.type())
    print('y_train:', y_train.size(), 'type:', y_train.type())
    break

X_train: torch.Size([32, 3, 32, 32]) type: torch.FloatTensor
y_train: torch.Size([32]) type: torch.LongTensor


In [ ]:
# 데이터 확인 2

pltsize = 1
plt.figure(figsize=(10 * pltsize, pltsize))

for i in range(10):
    plt.subplot(1, 10, i + 1)
    plt.axis('off')
    plt.imshow(np.transpose(X_train[i], (1, 2, 0)))
    plt.title('Class: ' + str(y_train[i].item()))


In [7]:
# CNN 모델 설계하기

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.conv1 = nn.Conv2d(   # Conv layer1
        in_channels = 3,
        out_channels = 8,
        kernel_size = 3,
        padding = 1)
        
        self.conv2 = nn.Conv2d(   # Conv layer2
        in_channels = 8,
        out_channels = 16,
        kernel_size = 3,
        padding = 1)
        
        self.pool = nn.MaxPool2d( # Pooling layer
        kernel_size = 2,
        stride = 2)
        
        self.fc1 = nn.Linear(8 * 8 * 16, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 10)

        
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)

        x = x.view(-1, 8 * 8 * 16)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = F.log_softmax(x)
        return x

In [8]:
# Optimizer, Objective Function 설정하기

model = CNN().to(DEVICE)                                      # 디바이스 설정
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
criterion = nn.CrossEntropyLoss()
#criterion = nn.NLLLoss()

print(model)

CNN(
  (conv1): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1024, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=10, bias=True)
)


---

In [ ]:
# MLP와 비교

class MLP(nn.Module):
    def __init__(self):                     # MLP클래스의 인스턴수를 생성했을 때 갖게 되는 성질을 정의하는 메소드
        super(MLP, self).__init__()         # nn.Module 내에 있는 메소드 상속받아 이용
        
        self.fc1 = nn.Linear(32*32*3, 512)  # 첫번째 fc layer 정의
        self.fc2 = nn.Linear(512, 256)      # 두번째 fc layer 정의
        self.fc3 = nn.Linear(256, 10)       # 세번째 fc layer 정의

        
    def forward(self, x):                   # 설계한 MLP모델에 데이터를 입력했을 때, Output까지의 계산 과정을 나열한 것
        x = x.view(-1, 32*32*3)             # MLP모델은 1차원 벡터값을 입력으로 받을 수 있기 때문에 (32*32*3)크기의 2차원 데이터를 view로 1차운으로 변환
        x = self.fc1(x)                     # fc layer1 통과
        x = F.relu(x)                       # 비선형 함수 ReLU() 계산 -> fc layer2의 input으로 계산
        x = self.fc2(x)                     # fc layer2 통과
        x = F.relu(x)                       # 비선형 함수 ReLU() 계산
        x = self.fc3(x)                     # fc layer3 통과
        x = F.log_softmax(x, dim=1)         # 10가지의 경우의 수 중 하나로 분류 (log_softmax() 사용 이유 : Back Propagation을 이용해 학습이 더 원활하게 진행하기 위해)
        return x

In [16]:
# 레이어를 쌓는 다른 방법 _ nn.Sequential 이용

class CNN2(nn.Module):
    def __init__(self):
        super(CNN2, self).__init__()
        
        self.feature_layer = nn.Sequential(
            
            nn.Conv2d(in_channels = 3, out_channels = 8, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2),
            
            nn.Conv2d(in_channels = 8, out_channels = 16, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2)
        
        )
        
        self.fc_layer = nn.Sequential(
            
            nn.Linear(8 * 8 * 16, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 10)
        
        )
        
        
    def forward(self, x):
        x = self.feature_layer(x)

        x = x.view(-1, 8 * 8 * 16)
        
        x = self.fc_layer(x)
        x = F.log_softmax(x)
        return x


In [17]:
# nn.Sequential 이용
# Optimizer, Objective Function 설정하기

model = CNN2().to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
criterion = nn.CrossEntropyLoss()

print(model)

CNN2(
  (feature_layer): Sequential(
    (0): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc_layer): Sequential(
    (0): Linear(in_features=1024, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=10, bias=True)
  )
)


---

In [9]:
# CNN 모델 학습을 진행하며 학습 데이터에 대한 모델 성능을 확인하는 함수 정의

def train(model, train_loader, optimizer, log_interval):
    model.train()
    for batch_idx, (image, label) in enumerate(train_loader):
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        optimizer.zero_grad()           # Optimizer의 Gradient를 초기화
        output = model(image)
        loss = criterion(output, label) # CrossEntropy를 이용해 Loss값 계산
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tTrain Loss: {:.6f}".format(
                epoch, batch_idx * len(image), 
                len(train_loader.dataset), 100. * batch_idx / len(train_loader), 
                loss.item()))

In [10]:
# 학습되는 과정 속에서 검증 데이터에 대한 모델 성능을 확인하는 함수 정의

def evaluate(model, test_loader):
    model.eval()                          # 평가 상태로 지정
    test_loss = 0
    correct = 0                           # 올바른 Class로 평가했는지 알아보기 위해

    with torch.no_grad():                 # Gradient를 통해 파라미터 값들이 업데이트되는 현상 방지
        for image, label in test_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            output = model(image)
            test_loss += criterion(output, label).item()     # test_loss값 업데이트
            prediction = output.max(1, keepdim = True)[1]
            correct += prediction.eq(label.view_as(prediction)).sum().item() # 예측 클래스와 실제 레이블 클래스 비교
    
    test_loss /= (len(test_loader.dataset) / BATCH_SIZE)
    test_accuracy = 100. * correct / len(test_loader.dataset)   # 정확도 계산
    return test_loss, test_accuracy

In [11]:
#log_soft + nllloss

for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, log_interval = 200)   # 정의한 train함수 실행
    test_loss, test_accuracy = evaluate(model, test_loader)     # 각 Epoch별로 출력되는 Loss값과 Accuracy 값 계산
    print("\n[EPOCH: {}], \tTest Loss: {:.4f}, \tTest Accuracy: {:.2f} % \n".format(
        epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/50000 (0%)]	Train Loss: 2.312438
Train Epoch: 1 [6400/50000 (13%)]	Train Loss: 1.964561
Train Epoch: 1 [12800/50000 (26%)]	Train Loss: 1.918607
Train Epoch: 1 [19200/50000 (38%)]	Train Loss: 1.766003
Train Epoch: 1 [25600/50000 (51%)]	Train Loss: 1.620650
Train Epoch: 1 [32000/50000 (64%)]	Train Loss: 1.369281
Train Epoch: 1 [38400/50000 (77%)]	Train Loss: 1.576364
Train Epoch: 1 [44800/50000 (90%)]	Train Loss: 1.267878

[EPOCH: 1], 	Test Loss: 1.4191, 	Test Accuracy: 48.25 % 

Train Epoch: 2 [0/50000 (0%)]	Train Loss: 1.413111
Train Epoch: 2 [6400/50000 (13%)]	Train Loss: 1.483805
Train Epoch: 2 [12800/50000 (26%)]	Train Loss: 1.709003
Train Epoch: 2 [19200/50000 (38%)]	Train Loss: 1.305506
Train Epoch: 2 [25600/50000 (51%)]	Train Loss: 1.504539
Train Epoch: 2 [32000/50000 (64%)]	Train Loss: 1.249981
Train Epoch: 2 [38400/50000 (77%)]	Train Loss: 1.629030
Train Epoch: 2 [44800/50000 (90%)]	Train Loss: 1.303033

[EPOCH: 2], 	Test Loss: 1.2829, 	Test Accuracy: 54.50 % 

KeyboardInterrupt: 

# Test Loss: 1.0687, 	Test Accuracy: 63.02 % 

In [16]:
# X + crossentropy

for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, log_interval = 200)   # 정의한 train함수 실행
    test_loss, test_accuracy = evaluate(model, test_loader)     # 각 Epoch별로 출력되는 Loss값과 Accuracy 값 계산
    print("\n[EPOCH: {}], \tTest Loss: {:.4f}, \tTest Accuracy: {:.2f} % \n".format(
        epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/50000 (0%)]	Train Loss: 2.350174
Train Epoch: 1 [6400/50000 (13%)]	Train Loss: 1.916218
Train Epoch: 1 [12800/50000 (26%)]	Train Loss: 1.745679
Train Epoch: 1 [19200/50000 (38%)]	Train Loss: 1.799946
Train Epoch: 1 [25600/50000 (51%)]	Train Loss: 1.620885
Train Epoch: 1 [32000/50000 (64%)]	Train Loss: 1.699204
Train Epoch: 1 [38400/50000 (77%)]	Train Loss: 1.344532
Train Epoch: 1 [44800/50000 (90%)]	Train Loss: 1.430129

[EPOCH: 1], 	Test Loss: 1.4862, 	Test Accuracy: 46.21 % 

Train Epoch: 2 [0/50000 (0%)]	Train Loss: 1.791456
Train Epoch: 2 [6400/50000 (13%)]	Train Loss: 1.197034
Train Epoch: 2 [12800/50000 (26%)]	Train Loss: 1.693008
Train Epoch: 2 [19200/50000 (38%)]	Train Loss: 1.860703
Train Epoch: 2 [25600/50000 (51%)]	Train Loss: 1.427170
Train Epoch: 2 [32000/50000 (64%)]	Train Loss: 1.468364
Train Epoch: 2 [38400/50000 (77%)]	Train Loss: 1.404573
Train Epoch: 2 [44800/50000 (90%)]	Train Loss: 1.495945

[EPOCH: 2], 	Test Loss: 1.3449, 	Test Accuracy: 49.95 % 

# Test Loss: 1.0128, 	Test Accuracy: 64.16 % 

In [21]:
# log_entropy + crossentropy

for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, log_interval = 200)   # 정의한 train함수 실행
    test_loss, test_accuracy = evaluate(model, test_loader)     # 각 Epoch별로 출력되는 Loss값과 Accuracy 값 계산
    print("\n[EPOCH: {}], \tTest Loss: {:.4f}, \tTest Accuracy: {:.2f} % \n".format(
        epoch, test_loss, test_accuracy))

<ipython-input-17-16503f65537c>:42: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.log_softmax(x)


Train Epoch: 1 [0/50000 (0%)]	Train Loss: 2.333009
Train Epoch: 1 [6400/50000 (13%)]	Train Loss: 2.076202
Train Epoch: 1 [12800/50000 (26%)]	Train Loss: 1.624784
Train Epoch: 1 [19200/50000 (38%)]	Train Loss: 2.140745
Train Epoch: 1 [25600/50000 (51%)]	Train Loss: 1.931095
Train Epoch: 1 [32000/50000 (64%)]	Train Loss: 1.710172
Train Epoch: 1 [38400/50000 (77%)]	Train Loss: 1.371194
Train Epoch: 1 [44800/50000 (90%)]	Train Loss: 1.400291

[EPOCH: 1], 	Test Loss: 1.4459, 	Test Accuracy: 47.43 % 

Train Epoch: 2 [0/50000 (0%)]	Train Loss: 1.612612
Train Epoch: 2 [6400/50000 (13%)]	Train Loss: 1.744985
Train Epoch: 2 [12800/50000 (26%)]	Train Loss: 1.355141
Train Epoch: 2 [19200/50000 (38%)]	Train Loss: 0.753993
Train Epoch: 2 [25600/50000 (51%)]	Train Loss: 0.937256
Train Epoch: 2 [32000/50000 (64%)]	Train Loss: 1.769789
Train Epoch: 2 [38400/50000 (77%)]	Train Loss: 1.153752
Train Epoch: 2 [44800/50000 (90%)]	Train Loss: 1.244892

[EPOCH: 2], 	Test Loss: 1.3248, 	Test Accuracy: 52.50 % 

# Test Loss: 1.0819, 	Test Accuracy: 61.93 % 

# log_soft + nllloss
## Test Loss: 1.0687, 	Test Accuracy: 63.02 % 

  

# X + crossentropy
## Test Loss: 1.0128, 	Test Accuracy: 64.16 %

  

# log_entropy + crossentropy
## Test Loss: 1.0819, 	Test Accuracy: 61.93 % 

In [15]:
# CNN 학습 실행하며 Train, Test set의 Loss 및 Test set Accuracy 확인하기

for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, log_interval = 200)   # 정의한 train함수 실행
    test_loss, test_accuracy = evaluate(model, test_loader)     # 각 Epoch별로 출력되는 Loss값과 Accuracy 값 계산
    print("\n[EPOCH: {}], \tTest Loss: {:.4f}, \tTest Accuracy: {:.2f} % \n".format(
        epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/50000 (0%)]	Train Loss: 2.301376
Train Epoch: 1 [6400/50000 (13%)]	Train Loss: 1.989344
Train Epoch: 1 [12800/50000 (26%)]	Train Loss: 1.743972
Train Epoch: 1 [19200/50000 (38%)]	Train Loss: 1.473368
Train Epoch: 1 [25600/50000 (51%)]	Train Loss: 1.869168
Train Epoch: 1 [32000/50000 (64%)]	Train Loss: 1.488392
Train Epoch: 1 [38400/50000 (77%)]	Train Loss: 1.576495
Train Epoch: 1 [44800/50000 (90%)]	Train Loss: 1.621193

[EPOCH: 1], 	Test Loss: 1.4529, 	Test Accuracy: 47.10 % 

Train Epoch: 2 [0/50000 (0%)]	Train Loss: 1.245101
Train Epoch: 2 [6400/50000 (13%)]	Train Loss: 1.455147
Train Epoch: 2 [12800/50000 (26%)]	Train Loss: 1.488896
Train Epoch: 2 [19200/50000 (38%)]	Train Loss: 1.498501
Train Epoch: 2 [25600/50000 (51%)]	Train Loss: 1.349244
Train Epoch: 2 [32000/50000 (64%)]	Train Loss: 1.437084
Train Epoch: 2 [38400/50000 (77%)]	Train Loss: 1.227584
Train Epoch: 2 [44800/50000 (90%)]	Train Loss: 1.697590

[EPOCH: 2], 	Test Loss: 1.3259, 	Test Accuracy: 52.81 % 

---

# Data Augmentation 기법 이용

In [18]:
# Data Augmentation가 적용된 CIFAR10 데이터 다운로드하기 (Train set, Test set 분리)

train_dataset = datasets.CIFAR10(root = "../data/CIFAR_10",
                                train = True,
                                download = True,
                                transform = transforms.Compose([                          # 괄호 안 처리 과정을 거친 데이터를 불러오는 것
                                    transforms.RandomHorizontalFlip(),                    # 해당 이미지를 50%확률로 좡우 반전하는 것
                                    transforms.ToTensor(),                                # Tensor 형태로 변환
                                    transforms.Normalize((0.5, 0.5, 0.5),                 # 다른 정규화 진행, RGB순으로 평균을 0.5씩 적용
                                                         (0.5, 0.5, 0.5))                 # 정규화 진행시 이용하는 표준편차, RGB순으로 0.5씩 적용
                                ]))

test_dataset = datasets.CIFAR10(root = "../data/CIFAR_10",
                                train = False,
                                transform = transforms.Compose([
                                    transforms.RandomHorizontalFlip(),
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))
                                ]))

train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = BATCH_SIZE,
                                          shuffle = True)

test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                         batch_size = BATCH_SIZE,
                                         shuffle = False)

Files already downloaded and verified


In [19]:
# 데이터 확인

for (X_train, y_train) in train_loader:
    print('X_train:', X_train.size(), 'type:', X_train.type())
    print('y_train:', y_train.size(), 'type:', y_train.type())
    break

X_train: torch.Size([32, 3, 32, 32]) type: torch.FloatTensor
y_train: torch.Size([32]) type: torch.LongTensor


In [20]:
# CNN 모델 설계하기

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(
        in_channels = 3,
        out_channels = 8,
        kernel_size = 3,
        padding = 1)
        
        self.conv2 = nn.Conv2d(
        in_channels = 8,
        out_channels = 16,
        kernel_size = 3,
        padding = 1)
        
        self.pool = nn.MaxPool2d(
        kernel_size = 2,
        stride = 2)
        
        self.fc1 = nn.Linear(8 * 8 * 16, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 10)

        
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)

        x = x.view(-1, 8 * 8 * 16)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = F.log_softmax(x)
        return x

In [21]:
# Optimizer, Objective Function 설정하기

model = CNN().to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
criterion = nn.CrossEntropyLoss()

print(model)

CNN(
  (conv1): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1024, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=10, bias=True)
)


In [22]:
# CNN 모델 학습을 진행하며 학습 데이터에 대한 모델 성능을 확인하는 함수 정의

def train(model, train_loader, optimizer, log_interval):
    model.train()
    for batch_idx, (image, label) in enumerate(train_loader):
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        optimizer.zero_grad()
        output = model(image)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tTrain Loss: {:.6f}".format(
                epoch, batch_idx * len(image), 
                len(train_loader.dataset), 100. * batch_idx / len(train_loader), 
                loss.item()))

In [23]:
# 학습되는 과정 속에서 검증 데이터에 대한 모델 성능을 확인하는 함수 정의

def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for image, label in test_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            output = model(image)
            test_loss += criterion(output, label).item()
            prediction = output.max(1, keepdim = True)[1]
            correct += prediction.eq(label.view_as(prediction)).sum().item()
    
    test_loss /= (len(test_loader.dataset) / BATCH_SIZE)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [24]:
# CNN 학습 실행하며 Train, Test set의 Loss 및 Test set Accuracy 확인하기

for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, log_interval = 200)
    test_loss, test_accuracy = evaluate(model, test_loader)
    print("\n[EPOCH: {}], \tTest Loss: {:.4f}, \tTest Accuracy: {:.2f} % \n".format(
        epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/50000 (0%)]	Train Loss: 2.292173
Train Epoch: 1 [6400/50000 (13%)]	Train Loss: 1.788858
Train Epoch: 1 [12800/50000 (26%)]	Train Loss: 1.694063
Train Epoch: 1 [19200/50000 (38%)]	Train Loss: 1.435321
Train Epoch: 1 [25600/50000 (51%)]	Train Loss: 1.367968
Train Epoch: 1 [32000/50000 (64%)]	Train Loss: 1.280787
Train Epoch: 1 [38400/50000 (77%)]	Train Loss: 1.424232
Train Epoch: 1 [44800/50000 (90%)]	Train Loss: 1.468520

[EPOCH: 1], 	Test Loss: 1.3683, 	Test Accuracy: 49.83 % 

Train Epoch: 2 [0/50000 (0%)]	Train Loss: 1.296031
Train Epoch: 2 [6400/50000 (13%)]	Train Loss: 1.220997
Train Epoch: 2 [12800/50000 (26%)]	Train Loss: 1.429867
Train Epoch: 2 [19200/50000 (38%)]	Train Loss: 1.164153
Train Epoch: 2 [25600/50000 (51%)]	Train Loss: 1.129965
Train Epoch: 2 [32000/50000 (64%)]	Train Loss: 1.190872
Train Epoch: 2 [38400/50000 (77%)]	Train Loss: 1.254067
Train Epoch: 2 [44800/50000 (90%)]	Train Loss: 1.118573

[EPOCH: 2], 	Test Loss: 1.1694, 	Test Accuracy: 58.46 % 

### CNN 모델 결과  
#### Test Loss: 1.0531, Test Accuracy: 62.75 %   

### Data Augmentation 기법 사용 결과  
#### Test Loss: 0.9456, Test Accuracy: 67.34 %   